In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
#
# ASE is a very convenient module for setting up simulations on molecules and 
# bulk materials
#
from ase.io import read, write
from ase.build import molecule
from ase.visualize import view
from ase.calculators.espresso import Espresso

In [ ]:
os.environ['ASE_ESPRESSO_COMMAND'] = "mpirun -np 4 /Users/oliviero/PWSCF/espresso-git/bin/pw.x -in PREFIX.pwi > PREFIX.pwo"
os.environ['OMP_NUM_THREADS'] = "1"

In [ ]:
pseudodir = '/Users/oliviero/PWSCF/espresso-git/pseudo'

In [ ]:
eV2Ry = 13.605662285137 # energy conversion factor
eV2kcal_mol = 23.0609 # energy conversion factor
bohr2ang = 0.5291772 # length conversion factor
ang2bohr = 1./bohr2ang

In [ ]:
atoms = molecule('H2O')
view(atoms, viewer="x3d")

In [ ]:
atoms.set_cell(15. * np.identity(3))
atoms.set_pbc((True, True, True))
atoms.center()

In [ ]:
pseudopotentials = {
    "H":"H.pbe-rrkjus.UPF",
    "O":"O.pbe-rrkjus.UPF"
}

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '/Users/oliviero/PWSCF/espresso-git/pseudo',
        'calculation': 'relax',
        'prefix': 'H2O_vacuum'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.4
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc

In [ ]:
energy = atoms.get_potential_energy()
print(f"Energy in vacuum = {energy:.3f} eV")

In [ ]:
relaxed_atoms = read('./espresso.pwo',format='espresso-out')

In [ ]:
atoms.positions

In [ ]:
relaxed_atoms.positions

# Use ASE for Geometry Optimization

In [ ]:
from ase.optimize import BFGS

In [ ]:
atoms = molecule('H2O')
atoms.set_cell(15. * np.identity(3))
atoms.set_pbc((True, True, True))
atoms.center()

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': pseudodir,
        'calculation': 'scf',
        'prefix': 'H2O_vacuum'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.4
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc

In [ ]:
optimization = BFGS(atoms,trajectory='H2O-opt.traj')
optimization.run(fmax=0.001)

In [ ]:
np.sqrt(np.sum((atoms.positions[0,:]-atoms.positions[1,:])**2))

In [ ]:
np.sqrt(np.sum((relaxed_atoms.positions[0,:]-relaxed_atoms.positions[1,:])**2))

# Use ASE for Phonons Calculations

In [ ]:
from ase.phonons import Phonons

In [ ]:
# Phonon calculator
N = 1
ph = Phonons(atoms, calc, supercell=(N, N, N), delta=0.05)
ph.run()


In [ ]:
ph.read(acoustic=True)
ph.clean()

In [ ]:
ph.C_N

In [ ]:

path = atoms.cell.bandpath('GX', npoints=100)
bs = ph.get_band_structure(path)

In [ ]:
dos = ph.get_dos(kpts=(20, 20, 20)).sample_grid(npts=100, width=1e-3)


In [ ]:

# Plot the band structure and DOS:
import matplotlib.pyplot as plt  # noqa

fig = plt.figure(1, figsize=(7, 4))
ax = fig.add_axes([.12, .07, .67, .85])

emax = 0.035
bs.plot(ax=ax, emin=0.0, emax=emax)

dosax = fig.add_axes([.8, .07, .17, .85])
dosax.fill_between(dos.get_weights(), dos.get_energies(), y2=0, color='grey',
                   edgecolor='k', lw=1)

dosax.set_ylim(0, emax)
dosax.set_yticks([])
dosax.set_xticks([])
dosax.set_xlabel("DOS", fontsize=18)


In [ ]:
dos.get_energies()

In [ ]:
from ase.build import molecule
from ase.calculators.emt import EMT
from ase.optimize import BFGS
from ase.vibrations import Vibrations

h2o = molecule('H2O')
h2o.calc = EMT()
BFGS(h2o).run(fmax=0.01)

vib = Vibrations(h2o)
vib.run()
vib.summary(log='H2O_EMT_summary.txt')
vib.write_mode(-1)

In [ ]:
vib.show_as_force(8)

In [ ]:
vib.write_mode(-1)

In [ ]:
trajectory = read('vib.8.traj')